In [1]:
from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler.passes import FullAncillaAllocation, EnlargeWithAncilla, ApplyLayout
from qiskit.converters import circuit_to_dag, dag_to_circuit
from utils import file_to_coupling_map, directory_to_circuits
import random
import time
import os
import numpy as np

In [2]:
seed = 42
random.seed(seed)

cmap_name= "heavy_hex_127.txt"
cmap = file_to_coupling_map(f"coupling_maps/{cmap_name}")
qc_name = "qft"
heuristic = "decay"
curr_seed = 42
qc_list = directory_to_circuits(f"circuits/{qc_name}")

pass_trials_list = sorted(set(np.logspace(0, 3, num=10, dtype=int)))

In [3]:
for qc in qc_list:
    print(f"{qc.name}")

qft_10.qasm
qft_29.qasm
qft_49.qasm
qft_68.qasm
qft_88.qasm
qft_107.qasm
qft_127.qasm


In [4]:
qc = qc_list[0]

num_qubits = qc.num_qubits
print(f"    Running {qc.name} ")

# CSV file to store the results
csv_path = f"results/trials_{qc_name}_10.csv"
# Columns are: "swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
with open(csv_path, "w") as f:
    f.write("swap_count,depth,runtime,num_qubits,heuristic,cmap,trials,seed\n")

    Running qft_10.qasm 


In [3]:
# Run the circuits for different number of trials
for pass_trials in pass_trials_list:
    print(f"Running {pass_trials} trials")
    # Create the sabre pass
    sabre_pass = SabreLayout(cmap, seed=curr_seed, max_iterations=4, swap_trials=pass_trials, layout_trials=pass_trials )
    dag = circuit_to_dag(qc)
    
    # Execute the pass manager
    start = time.time()
    dag_tr = sabre_pass.run(dag)
    end = time.time()
    # Collect the metrics for this trial
    runtime = end - start

    qc_tr = dag_to_circuit(dag_tr)
    swap_count = qc_tr.count_ops().get("swap", 0)
    depth = qc_tr.depth(lambda x: x.operation.num_qubits == 2)

    # Write the results to the CSV file
    # Columns are: "swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
    with open(csv_path, "a") as f:
        f.write(f"{swap_count},{depth},{runtime},{num_qubits},{heuristic},{cmap_name},{pass_trials},{curr_seed}\n")
    print(f"            Swap count: {swap_count}, Depth: {depth}, Runtime: {runtime}")

    Running qft_10.qasm 
Running 1 trials
            Swap count: 57, Depth: 54, Runtime: 0.11207699775695801
Running 2 trials
            Swap count: 57, Depth: 54, Runtime: 0.08727216720581055
Running 4 trials
            Swap count: 57, Depth: 54, Runtime: 0.07483386993408203
Running 6 trials
            Swap count: 57, Depth: 54, Runtime: 0.12169098854064941
Running 8 trials
            Swap count: 56, Depth: 56, Runtime: 0.1290419101715088
Running 12 trials
            Swap count: 54, Depth: 49, Runtime: 0.149733304977417
Running 18 trials
            Swap count: 54, Depth: 49, Runtime: 0.26051807403564453
Running 26 trials
            Swap count: 45, Depth: 54, Runtime: 0.4359290599822998
Running 37 trials
            Swap count: 45, Depth: 54, Runtime: 0.7678122520446777
Running 54 trials
            Swap count: 45, Depth: 54, Runtime: 1.3696069717407227
Running 78 trials
            Swap count: 45, Depth: 54, Runtime: 1.9821093082427979
Running 112 trials
            Swap count